## *Statistics coded:* Living conditions in Europe - income distribution and income inequality 

Prepared by [**Rodrigo González Santamaría**](rodrgo07@ucm.es) (EMOS).

This notebook aims at illustrating the *Statistics Explained* article on [Living conditions in Europe - income distribution and income inequality](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Living_conditions_in_Europe_-_income_distribution_and_income_inequality).


### Settings

Put all libraries/packages/modules/etc... that are necessary so that you can run this notebook.

In [6]:
from eurostatapiclient import EurostatAPIClient
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches

legend_properties = {'weight':'bold','size':'12'}
countries_names = {'AT':'Austria', 'BE':'Belgium', 'BG':'Bulgaria', 'CY': 'Cyprus', 'CZ': 'Czechia', 'DE': 'Germany', 'DK':
            'Denmark', 'EE':'Estonia', 'EL': 'Greece', 'ES':'Spain', 'FI':'Finland', 'FR':'France', 'HR':'Croatia', 
            'HU':'Hungary', 'IE':'Ireland', 'IT':'Italy', 'LT':'Lithuania', 'LU':'Luxembourg', 'LV':'Latvia', 'MT':
            'Malta', 'NL':'Netherlands', 'PL':'Poland', 'PT':'Portugal', 'RO':'Romania', 'SE':'Sweden', 'SI':'Slovenia',
            'SK':'Slovakia', 'UK':'United Kingdom', 'EU28': 'EU-28'}
countries_abbr = ['AT','BE','BG','CY','CZ','DE','DK','EE','EL','ES','FI',
                  'FR','HR','HU','IE','IT','LT','LU','LV','NV','NL','PL','PT','RO','SE','SI','SK','UK','MT', 'EU28']

In [7]:
#Set versions and formats, so far only the ones used here are availeable and call client
VERSION = 'v2.1'
FORMAT = 'json'
LANGUAGE = 'en'
client = EurostatAPIClient(VERSION, FORMAT, LANGUAGE)

### General overview
- **Key Findings**
- **Income distribution**
- **Income inequality**


### Documentation

### Key findings

*Ibid*, the code here will help reproducing the material presented in the section [Key findings](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Living_conditions_in_Europe_-_poverty_and_social_exclusion#Key_findings).

### Income distribution

*Ibid*, the code here will help reproducing the material presented in the section [Income distribution](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Living_conditions_in_Europe_-_income_distribution_and_income_inequality#Income_distribution).

**Map-1 Median equivalised disposable income in the EU-27**

In [ ]:
# We define params of dataframes
params_df1 = {
    'AGE': 'TOTAL',
    
}

### Income inequality

*Ibid*, the code here will help reproducing the material presented in the section [Income inequality](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Living_conditions_in_Europe_-_income_distribution_and_income_inequality#Income_inequality).